In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
from fuzzywuzzy import fuzz, process
import string
import itertools
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
#df_orig = pd.read_csv('data/csv_example_input_with_true_ids.csv')
df_orig = pd.read_csv('data/csv_example_output.csv')

In [4]:
df_orig.shape

(3337, 66)

In [5]:
df_orig.head(2)

,Cluster ID,confidence_score,Id,Source,Site name,Address,Zip,Phone,Fax,Program Name,...,canonical_Center Director,canonical_Funded Enrollment,canonical_Column2,canonical_Number per Site EHS,canonical_Neighborhood,canonical_Progmod,canonical_IDHS Provider ID,canonical_Number per Site HS,canonical_Executive Director,canonical_Id
0,0,0.993148,0,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Child Care,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
1,0,0.993148,1,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Child Care,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0


In [6]:
# Get entity blocks containing more than two records
# record_cnt = df_orig.groupby('Cluster ID').size()
# entity_ids = record_cnt[record_cnt > 2].index
# df_1 = df_orig.loc[df_orig['Cluster ID'].isin(entity_ids)].drop('confidence_score', axis=1)
df_1 = df_orig[df_orig.groupby('Cluster ID')['Cluster ID'].transform('size')>2]

### Records are merged into entity groups identified by Cluster ID

In [7]:
df_1.sort_values('Cluster ID').head(15)

,Cluster ID,confidence_score,Id,Source,Site name,Address,Zip,Phone,Fax,Program Name,...,canonical_Center Director,canonical_Funded Enrollment,canonical_Column2,canonical_Number per Site EHS,canonical_Neighborhood,canonical_Progmod,canonical_IDHS Provider ID,canonical_Number per Site HS,canonical_Executive Director,canonical_Id
0,0,0.993148,0,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Child Care,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
1226,0,0.993148,1226,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,State Pre-Kindergarten,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
1225,0,0.993148,1225,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,State Pre-Kindergarten,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
510,0,0.993148,510,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Head Start,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
509,0,0.993148,509,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Head Start,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
3255,0,0.969717,3255,purple_binder_early_childhood.csv,Salvation Army - Chicago Temple Corps Community Center,1 N Ogden Avenue,60607.0,2262649.0,NaN,NaN,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
215,0,0.987749,215,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army Temple,1 N. Ogden,NaN,2262649.0,NaN,Community Partnerships,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
2758,0,0.985435,2758,ECE Chicago Find a School scrape.csv,Salvation Army Temple,1 N. Ogden,60607.0,2262649.0,NaN,NaN,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
1879,0,0.999547,1879,chapin_dfss_providers_2011_070212.csv,SALVATION ARMY TEMPLE,1 N OGDEN,60640.0,2262649.0,NaN,NaN,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0
1,0,0.993148,1,CPS_Early_Childhood_Portal_scrape.csv,Salvation Army - Temple / Salvation Army,1 N Ogden Ave,NaN,2262649.0,NaN,Child Care,...,NaN,salvation army,child care,NaN,near west side,no,NaN,NaN,NaN,1.0


### Customizable scoring fields

In [8]:
unique_comb = ['Site name','Address','Program Name']

In [12]:
def check_nan(df, cols):
    #print 'Dataframe total row count: ', df.shape[0]
    result = []
    for c in cols:
        if sum(df[c].isnull())>0:
            #print c , ' has ' , sum(df[c].isnull()) , 'null/NAN values'
            #nan_cnt += 1
            result.append(c)
    return result

In [13]:
def clean(my_string):
    # Remove punctuations
    # Convert to lower cases
    return my_string.lower().translate(None, string.punctuation)

### Hefty scoing work is done in this function
### -- Unique value
###     -------- horizontal/vertical
### -- Pairwise comparison
###     -------- horizontal/vertical

In [9]:
# Apply to each entity group
def confidence_depricate(entity_group, solution, cols, weights=[0.4,0.2,0.4], axis=0):
    '''
    This function takes in a resolved/merged entity group, and generate
    a confidence score to indicate the uniformness of records in the group
    Note:
    Can't mix the score generating mechanism
    unique_val is indicating a rough proportion of unique values
    pair is indicating the fuzzy match score
    
    INPUT:
    entity_group: one entity group of merged records, type: pandas dataframe
    solution: score generating mechanism, type: string
    cols: selected record fields for score generating, type: string list
    weights: only used when axis=1, reflecting how important a column is, type: list
    axis: indicating row-wise or column-wise comparison, type: int
    
    OUTPUT:
    score: the confidence score, ranging from 0 to 1
    
    Example:
    confidence(entity_group, 'unique_vals', unique_comb)
    confidence(entity_group, 'unique_vals', unique_comb, axis=1)
    confidence(entity_group, 'pair', unique_comb
    '''
    total_records = entity_group.shape[0]
    # check nan cols
    non_cols = check_nan(entity_group, cols)
    #fill nan cols
    for nc in non_cols:
        entity_group.loc[:,nc]=entity_group[nc].fillna('VIA_filled')
    
    # Use unique value counts to generate confidence score
    if solution == 'unique_vals':
        result = []
        # read row by row
        if axis == 0:
            for idx, row in entity_group.iterrows():
                comb_str = []
                # call normalize function on each column
                # get a final combined string for the row
                for c in cols:
                    comb_str.append(clean(row[c]))
                # append the string to result set
                print ''.join(comb_str)
                result.append(''.join(comb_str))
            # Here we use 1-score to unify the score range with pairwise comparison
            # so that the higher the score is, the more confident we are
            score = 1 - len(set(result))*1./entity_group.shape[0]
            
        elif axis == 1:
            # this score is very sensitive because it takes the exact column value
            # when counting unique counts for EACH column
            score = 0
            for i, c in enumerate(cols):
                # unique count vs total row count in one column
                # weights can be customized
                print entity_group[c].values
                score += len(set(entity_group[c].apply(lambda x: clean(x)).values))*1./entity_group.shape[0]*weights[i]
            score = 1 - score
    # Pairwise Levenshtein distance score via fuzzywuzzy
    elif solution == 'pair':
        # Same as unique_val, combining all fields into one string
        # to be the comparison unit
        if axis == 0:
            result = []
            for idx, row in entity_group.iterrows():
                comb_str = []
                for c in cols:
                    comb_str.append(clean(row[c]))
                result.append(''.join(comb_str))
            # Get all possible record pairs
            scores = []
            for i, j in itertools.combinations(result, 2):
                scores.append(fuzz.ratio(i, j))
            score = (sum(scores)/(1.*len(scores)))/100
        # Generate vector of per column comparison
        elif axis == 1:
            # initialize result vector
            final_vector = np.zeros(len(unique_comb))
            # Get all possible record pairs
            pair_cnt = 0
            for i, j in itertools.combinations(entity_group[unique_comb].values, 2):
                pair_cnt += 1
                col_comp_vector = []
                # pairwise compare each col
                for idx, c in enumerate(unique_comb):
                    col_comp_vector.append(fuzz.ratio(i[idx],j[idx]))
                final_vector += col_comp_vector
            score = sum((final_vector/100/pair_cnt)*weights)
            
    elif solution == 'semi-pair':
        pass
    
    return round(score, 2)

In [10]:
def score_assign(entity_group, col_name, func, args):
    entity_group.loc[:, col_name] = func(entity_group, *args)
    return entity_group

### Score calculation formulus
Given:  
col1|col2  
a|2  
b|2  
b|2  
record_cnt = 3  
1. **Unique_val** - axis = 0  
unique(a2, b2, b2) = 2  
#convert for the higher the better convention  
score = 1 - 2/3 = 1/3 = 0.3333   
2. **Unique_val** - axis = 1  
weights = {col1:0.3, col2:0.7}  
unique_col1 = unique(a,b,b) = 2  
unique_col2 = unique(2,2,2) = 1  
score_col1 = 2/3  
score_col2 = 1/3  
score = 1 - [(2/3)*0.3 + (1/3)*0.7] = 0.56666  
3. **Pair** - axis = 0  
pair1 = (a2, b2)  
pair2 = (b2, b2)  
pair3 = (a2, b2)  
score1 = difflib.seq.ratio() = 0.5  
score2 = difflib.seq.ratio() = 1.0  
score3 = difflib.seq.ratio() = 0.5  
score = min(score1, score2, score3) = 0.5  
4. **Pair** - axis = 1  
pair1 = (a|2, b|2)  
pair2 = (b|2, b|2)  
pair3 = (a|2, b|2)  
score1 = sum([difflib.seq.ratio(a,b),difflib.seq.ratio(2,2)]* weights) = 
sum([0,1]*[0.3,0.7]) = 0.7  
score2 = sum([difflib.seq.ratio(b,b),difflib.seq.ratio(2,2)]* weights) = 
sum([1,1]*[0.3,0.7]) = 1  
score3 = sum([difflib.seq.ratio(a,b),difflib.seq.ratio(2,2)]* weights) = 
sum([0,1]*[0.3,0.7]) = 0.7  
score = min(score1, score2, score3) = 0.7  

### Dupe squash

In [11]:
def hash_gen(df, cols, key_col='md5_hash'):
    df.loc[:,key_col] = df[cols].apply(lambda x: hashlib.md5(x.to_string(header=False, index=False)).hexdigest(), axis=1)
    return df

In [12]:
def squash(entity_group, cols, key_col='md5_hash'):
    # group by hash
    # and form new entity_group by getting first row of each group
    new_entity_group = entity_group.groupby(key_col).first()
    new_entity_group.loc[:,'dupe_cnt'] = entity_group.groupby(key_col).size()
    new_entity_group.loc[:,'md5_hash'] = new_entity_group.index
    return new_entity_group

### Missing value handling
### weight redistribution formula
weights = [0.4, 0.3, 0.2, 0.1]
row = ['None', 'a', 'None', 'b']
so col1's weight needs to be redistributed to  
1) col2: 0.4*[0.3/(0.3+0.1)]  
2) col4: 0.4*[0.1/(0.3+0.1)]  
col3's weight needs to be redistributed to  
1) col2: 0.2*[0.3/(0.3+0.1)]
2) col4: 0.2*[0.1/(0.3+0.1)]
So  
new weight of col2 is 0.4*[0.3/(0.3+0.1)]+0.2*[0.3/(0.3+0.1)]+0.3 = 0.75  
new weight of col4 is 0.4*[0.1/(0.3+0.1)]+0.2*[0.1/(0.3+0.1)]+0.1 = 0.25  
  
So new_weights = [0, 0.75, 0, 0.25]  
  
For non-null field weight $Wi$ and null weight $Wj$:  
new weight for non-null field weight $Wi$ is:  
$Wi' = (Wi/\sum(Wi))*(\sum(Wj))+Wi$  

In [13]:
def weight_redistirbute(row_mask, weights):
    # Wi
    non_null_weights = np.array(weights)[~row_mask]
    # Wj
    null_weights = np.array(weights)[row_mask]
    
    new_weights = []
    for i,e in enumerate(row_mask):
        
        # For non-null weight
        if e == 0:
            new_w = (weights[i]/sum(non_null_weights))*sum(null_weights)+weights[i]
            new_weights.append(new_w)
        elif e == 1:
            new_weights.append(0)
    return new_weights

In [14]:
def confidence_depricate(entity_group, solution, cols, weights=[0.4,0.2,0.4], axis=0):
    '''
    This function takes in a resolved/merged entity group, and generate
    a confidence score to indicate the uniformness of records in the group
    Note:
    Can't mix the score generating mechanism
    unique_val is indicating a rough proportion of unique values
    pair is indicating the fuzzy match score
    
    INPUT:
    entity_group: one entity group of merged records, type: pandas dataframe
    solution: score generating mechanism, type: string
    cols: selected record fields for score generating, type: string list
    weights: only used when axis=1, reflecting how important a column is, type: list
    axis: indicating row-wise or column-wise comparison, type: int
    
    OUTPUT:
    score: the confidence score, ranging from 0 to 1
    
    Example:
    confidence(entity_group, 'unique_vals', unique_comb)
    confidence(entity_group, 'unique_vals', unique_comb, axis=1)
    confidence(entity_group, 'pair', unique_comb
    '''
    weights_orig = weights
    total_records = entity_group.shape[0]
    # check nan cols
    non_cols = check_nan(entity_group, cols)
    #fill nan cols
    for nc in non_cols:
        entity_group.loc[:,nc]=entity_group[nc].fillna('Z_filled')
    # remove duplicate rows
    entity_group = squash(entity_group, cols)
    # when all records are identical, which is a perfect merge
    if entity_group.shape[0]==1:
        return 1.
    
    # Use unique value counts to generate confidence score
    if solution == 'unique_vals':
        result = []
        # read row by row
        if axis == 0:
            for idx, row in entity_group.iterrows():
                comb_str = []
                # call normalize function on each column
                # get a final combined string for the row
                for c in cols:
                    comb_str.append(clean(row[c]))
                # append the string to result set
                #print ''.join(comb_str)
                result.append(''.join(comb_str))
            # Here we use 1-score to unify the score range with pairwise comparison
            # so that the higher the score is, the more confident we are
            score = 1 - len(set(result))*1./entity_group.shape[0]
            
        elif axis == 1:
            # this score is very sensitive because it takes the exact column value
            # when counting unique counts for EACH column
            score = 0
            for i, c in enumerate(cols):
                # unique count vs total row count in one column
                # weights can be customized
                print entity_group[c].values
                score += len(set(entity_group[c].apply(lambda x: clean(x)).values))*1./entity_group.shape[0]*weights[i]
            score = 1 - score
    # Pairwise Levenshtein distance score via fuzzywuzzy
    elif solution == 'pair':
        scores = []
        # Same as unique_val, combining all fields into one string
        # to be the comparison unit
        if axis == 0:
            result = []
            for idx, row in entity_group.iterrows():
                comb_str = []
                for c in cols:
                    comb_str.append(clean(row[c]))
                result.append(''.join(comb_str))
            # Get all possible record pairs
            for i, j in itertools.combinations(result, 2):
                scores.append(fuzz.ratio(i, j))
        # Generate vector of per column comparison
        elif axis == 1:
            # initialize result vector
            final_vector = np.zeros(len(unique_comb))
            # Get all possible record pairs
            pair_cnt = 0
            for i, j in itertools.combinations(entity_group[unique_comb].values, 2):
                # reset weights to original in the beginning of scoring
                weights = weights_orig
                # examine each record pair
                # handle possible missing value using WR
                mask_i, mask_j = (i=='Z_filled'), (j=='Z_filled')
                # mark any field with missing value in either i or j record, need to
                # redistribute all missing fields
                mask = mask_i|mask_j
                # if missing value exists, calculate new weights
                if sum(mask)>=1:
                    weights = weight_redistribute(mask, weights)
                pair_cnt += 1
                col_comp_vector = []
                # pairwise compare each col
                for idx, c in enumerate(unique_comb):
                    col_comp_vector.append(fuzz.ratio(i[idx],j[idx]))
                scores.append(sum([x*y for x,y in zip(col_comp_vector, weights)]))
        score = min(scores)  
        
    elif solution == 'semi-pair':
        pass
    
    return score

### Per record scoring

In [73]:
# Can't mix the score generating mechanism
# unique val is indicating a rough proportion of unique values, the lower the better
# pair is indicating the fuzzy match score, the higher the better
print 'Confidence score by unique row: \n', confidence(df_1[df_1['Cluster ID']==17], 'unique_vals', unique_comb)
print '\n'
print 'Confidence score by unique columns: \n', confidence(df_1[df_1['Cluster ID']==17], 'unique_vals', unique_comb, axis=1)
print '\n'
print 'Confidence score by pairwise comp: ', confidence(df_1[df_1['Cluster ID']==17], 'pair', unique_comb)
print '\n'
print 'Confidence score col-by-col by pairwise comp: ', confidence(df_1[df_1['Cluster ID']==17], 'pair', unique_comb, axis=1)


Confidence score by unique row: 
 south east asia center  southeast asia centerainslie1124 w ainslie st child care
 south east asia center  southeast asia centerainslie1124 w ainslie st school age
 south east asia center  southeast asia centerainslie1124 w ainslie st state prekindergarten
0.0


Confidence score by unique columns: 
[' South East Asia Center - South-East Asia Center(Ainslie)'
 ' South East Asia Center - South-East Asia Center(Ainslie)'
 ' South East Asia Center - South-East Asia Center(Ainslie)']
['1124 W Ainslie St ' '1124 W Ainslie St ' '1124 W Ainslie St ']
['Child Care' 'School Age' 'State Pre-Kindergarten']
0.4


Confidence score by pairwise comp:  0.9


Confidence score col-by-col by pairwise comp:  0.73


In [75]:
for id in entity_ids[:30]:
    score1 = confidence(df_1[df_1['Cluster ID']==id], 'pair', unique_comb)
    print 'Cluster Id: ', id
    print 'total record count: ',df_1[df_1['Cluster ID']==id].shape[0]
    print 'confidnece score: ', score1

Cluster Id:  0
total record count:  10
confidnece score:  0.74
Cluster Id:  2
total record count:  7
confidnece score:  0.87
Cluster Id:  4
total record count:  8
confidnece score:  0.8
Cluster Id:  7
total record count:  10
confidnece score:  0.91
Cluster Id:  8
total record count:  3
confidnece score:  0.81
Cluster Id:  9
total record count:  4
confidnece score:  0.95
Cluster Id:  10
total record count:  5
confidnece score:  0.88
Cluster Id:  11
total record count:  6
confidnece score:  0.77
Cluster Id:  12
total record count:  8
confidnece score:  0.75
Cluster Id:  13
total record count:  3
confidnece score:  0.84
Cluster Id:  14
total record count:  4
confidnece score:  0.87
Cluster Id:  15
total record count:  6
confidnece score:  0.68
Cluster Id:  17
total record count:  3
confidnece score:  0.9
Cluster Id:  18
total record count:  6
confidnece score:  0.92
Cluster Id:  20
total record count:  4
confidnece score:  0.88
Cluster Id:  21
total record count:  4
confidnece score:  0.9